# Taller #31: WebSockets e Interacción Visual en Tiempo Real
#### Desarrollado por: David Santiago Cruz Hernández

In [ ]:
import asyncio
import websockets
import json
import random
import math
from websockets.exceptions import ConnectionClosed

# pip install -r requirements.txt

## 1. Simulación de Coordenadas Aleatorias con WebSocket

### Generación de Coordenadas Aleatorias (incluyendo color)

In [ ]:
def generate_data():
    """
    Genera un conjunto de datos aleatorios con coordenadas x, y y un color.
    Returns:
        dict: Un diccionario con las coordenadas x, y y un color aleatorio.
    """

    return {
        "x": random.uniform(-5, 5),    # Coordenada x aleatoria entre -5 y 5
        "y": random.uniform(-5, 5),    # Coordenada y aleatoria entre -5 y 5
        "color": f"#{random.randint(0, 255):02X}{random.randint(0, 255):02X}{random.randint(0, 255):02X}"  # Color aleatorio en formato hexadecimal
    }

### Envío de Datos a través de WebSocket

In [ ]:
async def send_data(websocket):
    """
    Envía datos aleatorios al cliente conectado a través de WebSocket.
    Args:
        websocket (websockets.WebSocketServerProtocol): El objeto WebSocket del cliente conectado.
    """

    try:
        print("Cliente conectado para coordenadas")
        while True:
            data = generate_data()
            await websocket.send(json.dumps(data))
            await asyncio.sleep(0.5)
    except ConnectionClosed:
        print("Cliente desconectado para coordenadas")

### Servidor WebSocket

In [ ]:
async def main():
    """
    Inicia el servidor WebSocket en el puerto 8700.
    """
    print("Servidor WebSocket corriendo en ws://localhost:8700")
    async with websockets.serve(send_data, "localhost", 8700):
        await asyncio.Future()

<br>

---

<br>

## 2. Simulación de una Señal Cardíaca (ECG) con WebSocket

### Simulación de una señal cardíaca básica (onda ECG simple)

In [ ]:
def generate_ecg_signal(time):
    """
    Genera una señal ECG simulada basada en el tiempo.
    Args:
        time (float): El tiempo actual en segundos.
    Returns:
        float: El valor de la señal ECG en el tiempo dado, con ruido aleatorio.
    """
    # Onda R principal (pico alto)
    r_wave = math.sin(2 * math.pi * 1.5 * time) * 2  # Frecuencia cardíaca ~90 bpm
    r_spike = 1.5 * math.exp(-abs(5 * (time % 1 - 0.2))) if abs(time % 1 - 0.2) < 0.05 else 0

    # Otras ondas pequeñas (S y T)
    s_wave = -0.5 * math.sin(4 * math.pi * 1.5 * time + 0.5)
    t_wave = 0.6 * math.sin(3 * math.pi * 1.5 * time + 1.2)

    # Combinación de ondas
    total = r_wave + r_spike + s_wave + t_wave
    noise = random.uniform(-0.05, 0.05)

    return total + noise

### Generación de Datos ECG Simulados (incluyendo color)

In [ ]:
def generate_ecg_data(start_time):
    """
    Genera un conjunto de datos simulados de una señal ECG.
    Args:
        start_time (float): El tiempo de inicio en segundos desde el comienzo de la simulación.
    Returns:
        dict: Un diccionario con el valor de la señal ECG, el tiempo actual y un color aleatorio.
    """

    current_time = asyncio.get_event_loop().time() - start_time
    ecg_value = generate_ecg_signal(current_time)
    return {
        "ecg": ecg_value,
        "timestamp": current_time,
        "color": f"#{random.randint(0, 255):02X}{random.randint(0, 255):02X}{random.randint(0, 255):02X}"  # Color aleatorio en formato hexadecimal
    }

### Envío de Datos ECG a través de WebSocket

In [ ]:
async def send_ecg_data(websocket):
    """
    Envía datos de una señal ECG simulada al cliente conectado a través de WebSocket.
    Args:
        websocket (websockets.WebSocketServerProtocol): El objeto WebSocket del cliente conectado.
    """
    start_time = asyncio.get_event_loop().time()
    try:
        print("Cliente conectado para ECG")
        while True:
            data = generate_ecg_data(start_time)
            await websocket.send(json.dumps(data))
            await asyncio.sleep(0.01)  # Enviar datos más frecuentemente para una onda suave
    except ConnectionClosed:
        print("Cliente desconectado de ECG")

### Servidor WebSocket para la señal ECG

In [ ]:
async def main_ecg():
    """
    Inicia el servidor WebSocket para la señal ECG en el puerto 8800.
    """
    print("Servidor WebSocket corriendo en ws://localhost:8800")
    async with websockets.serve(send_ecg_data, "localhost", 8800):
        await asyncio.Future()

## 3. Ejecución de ambos servidores WebSocket

- Ejecuta ambos servidores WebSocket (coordenadas y ECG) de manera concurrente.
- Esta función utiliza asyncio.gather para ejecutar ambas tareas en paralelo.

In [ ]:
async def run_both():
    await asyncio.gather(
        main(),
        main_ecg()
    )

asyncio.run(await run_both())